In [1]:
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# ───────────────────────────────────────────────────────────────
# CONFIGURACIÓN
# ───────────────────────────────────────────────────────────────

BASE_PATH    = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\Conjunt de dades Preprocessades\Datasets"
RESULTS_PATH = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM"
DATASETS     = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv"
]

N_STEPS = 30
FEATURE_COLUMNS = [
    "Open","High","Low","Volume",
    "EMA_7","EMA_40","MACD","Signal_Line",
    "MACD_Hist","RSI","ATR"
]
TARGET_COLUMN = "Close"
TEST_RATIO = 0.10
VAL_RATIO  = 0.10
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO


# ───────────────────────────────────────────────────────────────
# FUNCIONES AUXILIARES
# ───────────────────────────────────────────────────────────────

def create_sequences(X, y, n_steps=30):
    Xs, ys = [], []
    for i in range(n_steps, len(X)):
        Xs.append(X[i - n_steps:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def build_lstm_model(sequence_length, n_features, units, n_layers, dropout, learning_rate):
    """
    Reconstruye la arquitectura LSTM con los hiperparámetros dados.
    """
    model = Sequential()
    for i in range(n_layers):
        is_first = (i == 0)
        is_last = (i == n_layers - 1)
        return_sequences = not is_last
        if is_first:
            model.add(LSTM(
                units, return_sequences=return_sequences,
                input_shape=(sequence_length, n_features)
            ))
        else:
            model.add(LSTM(units, return_sequences=return_sequences))
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="huber", optimizer=optimizer, metrics=["mean_absolute_error"])
    return model

def compute_metrics(model, X_scaled, y_scaled, scaler_y):
    """
    Calcula MAE, RMSE, R² y devuelve y_true, y_pred.
    """
    y_pred_scaled = model.predict(X_scaled, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2, y_true, y_pred

def recompute_indicators(df):
    """
    Recalcula EMA_7, EMA_40, MACD, Signal_Line, MACD_Hist, RSI, ATR para todo df.
    """
    close = df['Close']
    df['EMA_7'] = close.ewm(span=7, adjust=False).mean()
    df['EMA_40'] = close.ewm(span=40, adjust=False).mean()

    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    signal = macd.ewm(span=9, adjust=False).mean()
    df['MACD'] = macd
    df['Signal_Line'] = signal
    df['MACD_Hist'] = macd - signal

    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df['RSI'] = 100 - (100 / (1 + rs))

    df['ATR'] = (df['High'] - df['Low']).rolling(window=14).mean()


# ───────────────────────────────────────────────────────────────
# BUCLE PRINCIPAL: CARGAR CADA DATASET, LEER PESOS, GUARDAR MÉTRICAS Y GRAFICAR
# ───────────────────────────────────────────────────────────────

for filename in DATASETS:
    dataset_name = os.path.splitext(filename)[0]
    print(f"\n===== Procesando: {dataset_name} =====")

    # Rutas
    data_path    = os.path.join(BASE_PATH, filename)
    model_folder = os.path.join(RESULTS_PATH, dataset_name)
    weights_path = os.path.join(model_folder, f"{dataset_name}_best_weights.weights.h5")
    params_path  = os.path.join(model_folder, f"{dataset_name}_best_params.json")

    if not os.path.isfile(weights_path) or not os.path.isfile(params_path):
        print(f"  ⚠️ Faltan pesos o parámetros para {dataset_name}, omitiendo.")
        continue

    # 1) Cargar CSV original
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values("Date", inplace=True)
    df = df.dropna(subset=FEATURE_COLUMNS + [TARGET_COLUMN]).reset_index(drop=True)

    # 2) Crear secuencias y dividir en train/val/test
    data_X = df[FEATURE_COLUMNS].values
    data_y = df[TARGET_COLUMN].values.reshape(-1, 1)
    X_seq, y_seq = create_sequences(data_X, data_y, n_steps=N_STEPS)

    n_total = len(X_seq)
    train_end = int(n_total * TRAIN_RATIO)
    val_end   = train_end + int(n_total * VAL_RATIO)

    X_train = X_seq[:train_end]
    y_train = y_seq[:train_end]

    X_val = X_seq[train_end:val_end]
    y_val = y_seq[train_end:val_end]

    X_test = X_seq[val_end:]
    y_test = y_seq[val_end:]

    # 3) Reconstruir scalers según train+val
    scaler_X = MinMaxScaler()
    X_train_val_flat = X_seq[:val_end].reshape(-1, len(FEATURE_COLUMNS))
    scaler_X.fit(X_train_val_flat)

    scaler_y = MinMaxScaler()
    scaler_y.fit(y_seq[:val_end])

    def scale_X(X):
        flat = X.reshape(-1, len(FEATURE_COLUMNS))
        flat_scaled = scaler_X.transform(flat)
        return flat_scaled.reshape(X.shape)

    X_train_scaled = scale_X(X_train)
    X_val_scaled   = scale_X(X_val)
    X_test_scaled  = scale_X(X_test)

    y_train_scaled = scaler_y.transform(y_train)
    y_val_scaled   = scaler_y.transform(y_val)
    y_test_scaled  = scaler_y.transform(y_test)

    # 4) Cargar hiperparámetros
    with open(params_path, 'r') as f:
        best_params = json.load(f)
    n_layers      = best_params['n_layers']
    units         = best_params['units']
    dropout       = best_params['dropout']
    learning_rate = best_params['learning_rate']

    print(f"  ✓ Parámetros cargados: layers={n_layers}, units={units}, dropout={dropout}, lr={learning_rate}")

    # 5) Reconstruir modelo y cargar pesos
    model = build_lstm_model(
        sequence_length=N_STEPS,
        n_features=len(FEATURE_COLUMNS),
        units=units,
        n_layers=n_layers,
        dropout=dropout,
        learning_rate=learning_rate
    )
    model.load_weights(weights_path)
    print(f"  ✓ Pesos cargados desde: {weights_path}")

    # 6) Calcular métricas en Test
    mae_test, rmse_test, r2_test, y_true, y_pred = compute_metrics(
        model, X_test_scaled, y_test_scaled, scaler_y
    )
    print(f"  → Test MAE={mae_test:.4f}, RMSE={rmse_test:.4f}, R²={r2_test:.4f}")

    # 6.1) Guardar métricas en CSV
    df_metrics = pd.DataFrame({
        "Dataset": [dataset_name],
        "MAE":     [mae_test],
        "RMSE":    [rmse_test],
        "R2":      [r2_test]
    })
    metrics_csv = os.path.join(model_folder, f"{dataset_name}_metrics_test.csv")
    df_metrics.to_csv(metrics_csv, index=False)
    print(f"  ✓ Métricas guardadas en: {metrics_csv}")

    # 7) Graficar Real vs Predicho (Test)
    # --------------------------------------------------
    # Primer, calculamos el índice donde empieza el test en df['Date']:
    start_test_idx = N_STEPS + val_end
    dates_test = df['Date'].iloc[start_test_idx : start_test_idx + len(y_true)].reset_index(drop=True)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_true.flatten(),
        mode='lines',
        name='Real (Close)',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_pred.flatten(),
        mode='lines',
        name='Predit',
        line=dict(color='red', dash='dash')
    ))
    fig.update_layout(
        title=f"{dataset_name} – Real vs Predicció (Test, LSTM)",
        xaxis_title='Data',
        yaxis_title='Preu (Close)',
        template='plotly_white',
        xaxis_rangeslider_visible=True
    )
    plot_html = os.path.join(model_folder, f"{dataset_name}_test_plot.html")
    fig.write_html(plot_html)
    print(f"  ✓ Gráfica Test guardada en: {plot_html}")
    # --------------------------------------------------

    # 8) Predicción autoregresiva de los próximos 10 días laborables
    df_future = df.copy().reset_index(drop=True)
    recompute_indicators(df_future)

    last_sequence = X_test_scaled[-1].copy().reshape(1, N_STEPS, len(FEATURE_COLUMNS))
    future_preds = []
    future_dates = pd.bdate_range(start=df_future['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)

    for date in future_dates:
        # 8.1) Predicción escalada y remapeo a valor real
        y_pred_scaled = model.predict(last_sequence, verbose=0)[0][0]
        y_pred_real = scaler_y.inverse_transform([[y_pred_scaled]])[0][0]
        future_preds.append(y_pred_real)

        # 8.2) Añadir la predicción al DataFrame para recalcular indicadores
        prev = df_future.iloc[-1]
        new_row = {
            'Date':        date,
            'Open':        prev['Close'],   # asumimos que Open = Close anterior
            'High':        y_pred_real,
            'Low':         y_pred_real,
            'Volume':      prev['Volume'],  # mantenemos igual
            'Close':       y_pred_real,
            'EMA_7':       np.nan,
            'EMA_40':      np.nan,
            'MACD':        np.nan,
            'Signal_Line': np.nan,
            'MACD_Hist':   np.nan,
            'RSI':         np.nan,
            'ATR':         np.nan
        }
        df_future.loc[len(df_future)] = new_row
        recompute_indicators(df_future)

        # 8.3) Extraer las últimas N_STEPS filas de indicadores técnicos
        recent_features = df_future[FEATURE_COLUMNS].iloc[-N_STEPS:].values
        recent_scaled = scaler_X.transform(recent_features)
        last_sequence = recent_scaled.reshape(1, N_STEPS, len(FEATURE_COLUMNS))

    # 9) Guardar predicciones futuras en CSV
    df_fut_pred = pd.DataFrame({
        "Date": future_dates,
        "Predicted_Close": future_preds
    })
    fut_csv = os.path.join(model_folder, f"{dataset_name}_future_10days.csv")
    df_fut_pred.to_csv(fut_csv, index=False)
    print(f"  ✓ Predicciones futuras guardadas en: {fut_csv}")

    # 10) Graficar histórico + predicciones futuras
    fig_future = go.Figure()
    fig_future.add_trace(go.Scatter(
        x=df['Date'], y=df['Close'],
        mode='lines', name='Històric Close', line=dict(color='lightblue')
    ))
    fig_future.add_trace(go.Scatter(
        x=future_dates, y=np.array(future_preds),
        mode='lines+markers', name='Predicción futura',
        line=dict(color='orange', dash='dash'),
        marker=dict(size=6)
    ))
    fig_future.update_layout(
        title=f"{dataset_name} – Predicción Pròxims 10 dies",
        xaxis_title='Data',
        yaxis_title='Preu (Close)',
        template='plotly_white',
        xaxis_rangeslider_visible=True
    )
    fut_html = os.path.join(model_folder, f"{dataset_name}_future_plot.html")
    fig_future.write_html(fut_html)
    print(f"  ✓ Gráfica futura guardada en: {fut_html}")



===== Procesando: Amazon_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, units=512, dropout=0.1, lr=0.001


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  ✓ Pesos cargados desde: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_best_weights.weights.h5
  → Test MAE=4.5752, RMSE=5.7316, R²=0.9283
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_plot.html

===== Procesando: Euro_Stoxx_50_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, uni

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



  → Test MAE=51.0807, RMSE=64.9371, R²=0.7725
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_plot.html

===== Procesando: Google_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, units=512, dropout=0.5, lr=0.001


...

  ✓ Pesos cargados desde: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Indra_Stock_Price_output\Indra_Stock_Price_output_best_weights.weights.h5
  → Test MAE=234.5538, RMSE=313.4665, R²=0.7487
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Indra_Stock_Price_output\Indra_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Indra_Stock_Price_output\Indra_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Indra_Stock_Price_output\Indra_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Indra_Stock_Price_output\Indra_Stock_Price_output_future_plot.html

===== Procesando: P&G_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, units=256, dropout=

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



  → Test MAE=2.1416, RMSE=2.6764, R²=0.6456
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_future_plot.html

===== Procesando: S&P500_Stock_Price_output =====
  ✓ Parámetros cargados: layers=2, units=128, dropout=0.1, lr=0.001
  ✓ Pesos cargados desde: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_best_weights.weights.h5


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 



  → Test MAE=71.1080, RMSE=90.2894, R²=0.8303
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_plot.html


GENERACIÓ MODEL HÍBRID: LSTM + XGBOOST

In [1]:
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# ───────────────────────────────────────────────────────────────
# AFEGIM IMPORTS PER XGBOOST
# ───────────────────────────────────────────────────────────────
import xgboost as xgb
from sklearn.model_selection import train_test_split

# ───────────────────────────────────────────────────────────────
# CONFIGURACIÓ DE RUTES
# ───────────────────────────────────────────────────────────────

# Carpeta on estan els datasets CSV
BASE_PATH = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\Conjunt de dades Preprocessades\Datasets"

# Carpeta base on estan les subcarpetes amb pesos i paràmetres (per carregar el model LSTM)
WEIGHTS_PATH_BASE = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM"

# Carpeta on desitgem guardar tots els resultats (metrics, gràfics, futures prediccions)
RESULTS_PATH = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID"

# Llista de fitxers CSV a processar
DATASETS = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv"
]

# ───────────────────────────────────────────────────────────────
# HIPERPARÀMETRES I PARAMETRES
# ───────────────────────────────────────────────────────────────

N_STEPS = 30
FEATURE_COLUMNS = [
    "Open","High","Low","Volume",
    "EMA_7","EMA_40","MACD","Signal_Line",
    "MACD_Hist","RSI","ATR"
]
TARGET_COLUMN = "Close"
TEST_RATIO = 0.10
VAL_RATIO  = 0.10
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO

# ───────────────────────────────────────────────────────────────
# FUNCIONS AUXILIARS
# ───────────────────────────────────────────────────────────────

def create_sequences(X, y, n_steps=30):
    Xs, ys = [], []
    for i in range(n_steps, len(X)):
        Xs.append(X[i - n_steps:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def build_lstm_model(sequence_length, n_features, units, n_layers, dropout, learning_rate):
    model = Sequential()
    for i in range(n_layers):
        is_first = (i == 0)
        is_last = (i == n_layers - 1)
        return_sequences = not is_last
        if is_first:
            model.add(LSTM(
                units, return_sequences=return_sequences,
                input_shape=(sequence_length, n_features)
            ))
        else:
            model.add(LSTM(units, return_sequences=return_sequences))
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="huber", optimizer=optimizer, metrics=["mean_absolute_error"])
    return model

def compute_metrics(model, X_scaled, y_scaled, scaler_y):
    y_pred_scaled = model.predict(X_scaled, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2, y_true, y_pred

def recompute_indicators(df):
    close = df['Close']
    df['EMA_7'] = close.ewm(span=7, adjust=False).mean()
    df['EMA_40'] = close.ewm(span=40, adjust=False).mean()

    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    signal = macd.ewm(span=9, adjust=False).mean()
    df['MACD'] = macd
    df['Signal_Line'] = signal
    df['MACD_Hist'] = macd - signal

    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df['RSI'] = 100 - (100 / (1 + rs))

    df['ATR'] = (df['High'] - df['Low']).rolling(window=14).mean()


# ───────────────────────────────────────────────────────────────
# BUCLE PRINCIPAL: CÀRREGA, LSTM, XGBOOST HÍBRID I GRAFICS
# ───────────────────────────────────────────────────────────────

for filename in DATASETS:
    dataset_name = os.path.splitext(filename)[0]
    print(f"\n===== Procesant: {dataset_name} =====")

    # ───────────────────────────────────────────────────────────
    # 1) DEFINICIÓ DE RUTES DE PESOS I PARÀMETRES LSTM
    # ───────────────────────────────────────────────────────────
    weights_folder = os.path.join(WEIGHTS_PATH_BASE, dataset_name)
    weights_path   = os.path.join(weights_folder, f"{dataset_name}_best_weights.weights.h5")
    params_path    = os.path.join(weights_folder, f"{dataset_name}_best_params.json")

    # ───────────────────────────────────────────────────────────
    # 2) DEFINICIÓ DE LA CARPETA DE RESULTATS
    # ───────────────────────────────────────────────────────────
    results_folder = os.path.join(RESULTS_PATH, dataset_name)
    os.makedirs(results_folder, exist_ok=True)

    if not os.path.isfile(weights_path) or not os.path.isfile(params_path):
        print(f"  ⚠️  Falten pesos o paràmetres per a {dataset_name}, s’omet.")
        continue

    # ───────────────────────────────────────────────────────────
    # 3) LLEGIR CSV ORIGINAL
    # ───────────────────────────────────────────────────────────
    data_path = os.path.join(BASE_PATH, filename)
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values("Date", inplace=True)
    df = df.dropna(subset=FEATURE_COLUMNS + [TARGET_COLUMN]).reset_index(drop=True)

    # ───────────────────────────────────────────────────────────
    # 4) CREAR SEQÜÈNCIES AMB LES DADES BRUTES (per a XGBoost)
    # ───────────────────────────────────────────────────────────
    data_X_raw = df[FEATURE_COLUMNS].values      # dades sense escalar
    data_y_raw = df[TARGET_COLUMN].values.reshape(-1, 1)
    X_seq_raw, y_seq_raw = create_sequences(data_X_raw, data_y_raw, n_steps=N_STEPS)

    # ───────────────────────────────────────────────────────────
    # 5) CREAR SEQÜÈNCIES PER A LSTM I DIVIDIR EN TRAIN/VAL/TEST
    # ───────────────────────────────────────────────────────────
    data_X = data_X_raw.copy()
    data_y = data_y_raw.copy()
    X_seq, y_seq = create_sequences(data_X, data_y, n_steps=N_STEPS)

    n_total = len(X_seq)
    train_end = int(n_total * TRAIN_RATIO)
    val_end   = train_end + int(n_total * VAL_RATIO)

    # Seqüències per a LSTM
    X_train = X_seq[:train_end]
    y_train = y_seq[:train_end]
    X_val   = X_seq[train_end:val_end]
    y_val   = y_seq[train_end:val_end]
    X_test  = X_seq[val_end:]
    y_test  = y_seq[val_end:]

    # Seqüències brutes (raw) per a features XGBoost (només el darrer pas de cada seqüència)
    X_seq_raw_train = X_seq_raw[:train_end]
    X_seq_raw_val   = X_seq_raw[train_end:val_end]
    X_seq_raw_test  = X_seq_raw[val_end:]

    # ───────────────────────────────────────────────────────────
    # 6) ESCALAT DE CARACTERÍSTIQUES (LSTM)
    # ───────────────────────────────────────────────────────────
    scaler_X = MinMaxScaler()
    X_train_val_flat = X_seq[:val_end].reshape(-1, len(FEATURE_COLUMNS))
    scaler_X.fit(X_train_val_flat)

    scaler_y = MinMaxScaler()
    scaler_y.fit(y_seq[:val_end])

    def scale_X(X):
        flat = X.reshape(-1, len(FEATURE_COLUMNS))
        flat_scaled = scaler_X.transform(flat)
        return flat_scaled.reshape(X.shape)

    X_train_scaled = scale_X(X_train)
    X_val_scaled   = scale_X(X_val)
    X_test_scaled  = scale_X(X_test)

    y_train_scaled = scaler_y.transform(y_train)
    y_val_scaled   = scaler_y.transform(y_val)
    y_test_scaled  = scaler_y.transform(y_test)

    # ───────────────────────────────────────────────────────────
    # 7) CARREGAR HIPERPARÀMETRES LSTM
    # ───────────────────────────────────────────────────────────
    with open(params_path, 'r') as f:
        best_params = json.load(f)
    n_layers      = best_params['n_layers']
    units         = best_params['units']
    dropout       = best_params['dropout']
    learning_rate = best_params['learning_rate']

    print(f"  ✓ Paràmetres LSTM carregats: layers={n_layers}, units={units}, dropout={dropout}, lr={learning_rate}")

    # ───────────────────────────────────────────────────────────
    # 8) RECONSTRUIR MODEL LSTM I CARREGAR PESOS
    # ───────────────────────────────────────────────────────────
    model = build_lstm_model(
        sequence_length=N_STEPS,
        n_features=len(FEATURE_COLUMNS),
        units=units,
        n_layers=n_layers,
        dropout=dropout,
        learning_rate=learning_rate
    )
    model.load_weights(weights_path)
    print(f"  ✓ Pesos LSTM carregats de: {weights_path}")

    # ───────────────────────────────────────────────────────────
    # 9) AVALUAR LSTM SOBRE TEST
    # ───────────────────────────────────────────────────────────
    mae_test, rmse_test, r2_test, y_true_test, y_pred_test = compute_metrics(
        model, X_test_scaled, y_test_scaled, scaler_y
    )
    print(f"  → LSTM Test MAE={mae_test:.4f}, RMSE={rmse_test:.4f}, R²={r2_test:.4f}")

    # 9.1) Guardar mètriques LSTM
    df_metrics = pd.DataFrame({
        "Dataset": [dataset_name],
        "MAE_LSTM": [mae_test],
        "RMSE_LSTM": [rmse_test],
        "R2_LSTM": [r2_test]
    })
    metrics_csv = os.path.join(results_folder, f"{dataset_name}_metrics_LSTM_test.csv")
    df_metrics.to_csv(metrics_csv, index=False)
    print(f"  ✓ Mètriques LSTM guardades a: {metrics_csv}")

    # ───────────────────────────────────────────────────────────
    # 10) XGB-HÍBRID: ENTRENAR XGBOOST SOBRE ELS RESIDUALS DE LA LSTM
    # ───────────────────────────────────────────────────────────

    # 10.1) Prediccions LSTM sobre train+val per calcular residuals
    X_train_val_scaled = np.concatenate([X_train_scaled, X_val_scaled], axis=0)
    y_train_val_scaled = np.concatenate([y_train_scaled, y_val_scaled], axis=0)

    y_train_val_pred_scaled = model.predict(X_train_val_scaled, verbose=0)
    y_train_val_pred_real = scaler_y.inverse_transform(y_train_val_pred_scaled)
    y_train_val_real = scaler_y.inverse_transform(y_train_val_scaled)

    residuals_train_val = (y_train_val_real.flatten() - y_train_val_pred_real.flatten())

    # 10.2) Crear features per a XGBoost: darrer pas de seqüència raw + predicció LSTM
    X_seq_raw_train_val = np.concatenate([X_seq_raw_train, X_seq_raw_val], axis=0)
    X_last_train_val = X_seq_raw_train_val[:, -1, :]  # forma: (n_samples, n_features)

    XGB_features_train_val = np.concatenate([
        X_last_train_val,
        y_train_val_pred_real.reshape(-1, 1)
    ], axis=1)  # (n_samples, n_features+1)

    # 10.3) Entrenar XGBoost regressor sobre els residuals
    X_xgb_train, X_xgb_val, y_xgb_train, y_xgb_val = train_test_split(
        XGB_features_train_val, residuals_train_val, test_size=0.2, random_state=42
    )

    xgb_model = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    xgb_model.fit(
        X_xgb_train, y_xgb_train,
        eval_set=[(X_xgb_val, y_xgb_val)],
        verbose=False
    )
    print(f"  ✓ XGBoost entrenat sobre residuals (train+val) per a {dataset_name}")

    # ───────────────────────────────────────────────────────────
    # 11) AVALUAR HÍBRID SOBRE TEST
    # ───────────────────────────────────────────────────────────

    # 11.1) Obtenir prediccions LSTM sobre Test (invertides)
    y_pred_lstm_test = y_pred_test.flatten()

    # 11.2) Preparar features XGBoost per test: darrer pas seqüència raw + pred LSTM
    X_last_test = X_seq_raw_test[:, -1, :]  # (n_test, n_features)
    XGB_features_test = np.concatenate([
        X_last_test,
        y_pred_lstm_test.reshape(-1, 1)
    ], axis=1)

    # 11.3) Predir residuals amb XGBoost
    residuals_pred_test = xgb_model.predict(XGB_features_test)

    # 11.4) Predicció híbrida final = pred LSTM + pred residual
    y_pred_hybrid_test = y_pred_lstm_test + residuals_pred_test

    # 11.5) Calcular mètriques híbrides sobre Test
    y_true_hybrid_test = y_true_test.flatten()
    mae_hybrid = mean_absolute_error(y_true_hybrid_test, y_pred_hybrid_test)
    rmse_hybrid = np.sqrt(mean_squared_error(y_true_hybrid_test, y_pred_hybrid_test))
    r2_hybrid = r2_score(y_true_hybrid_test, y_pred_hybrid_test)
    print(f"  → HÍBRID Test MAE={mae_hybrid:.4f}, RMSE={rmse_hybrid:.4f}, R²={r2_hybrid:.4f}")

    # 11.6) Guardar mètriques híbrides
    df_metrics_hybrid = pd.DataFrame({
        "Dataset": [dataset_name],
        "MAE_HYBRID": [mae_hybrid],
        "RMSE_HYBRID": [rmse_hybrid],
        "R2_HYBRID": [r2_hybrid]
    })
    metrics_hybrid_csv = os.path.join(results_folder, f"{dataset_name}_metrics_hybrid_test.csv")
    df_metrics_hybrid.to_csv(metrics_hybrid_csv, index=False)
    print(f"  ✓ Mètriques híbrides guardades a: {metrics_hybrid_csv}")

    # ───────────────────────────────────────────────────────────
    # 12) GRAFICA LSTM vs HÍBRID (Test)
    # ───────────────────────────────────────────────────────────
    start_test_idx = N_STEPS + val_end
    dates_test = df['Date'].iloc[start_test_idx : start_test_idx + len(y_true_test)].reset_index(drop=True)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_true_test.flatten(),
        mode='lines',
        name='Real (Close)',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_pred_test.flatten(),
        mode='lines',
        name='Pred LSTM',
        line=dict(color='red', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=dates_test,
        y=y_pred_hybrid_test,
        mode='lines',
        name='Pred HÍBRIDA',
        line=dict(color='orange', dash='dot')
    ))
    fig.update_layout(
        title=f"{dataset_name} – Real vs Prediccions (LSTM i HÍBRIDA) (Test)",
        xaxis_title='Data',
        yaxis_title='Preu Close (USD)',
        template='plotly_white',
        xaxis_rangeslider_visible=True
    )
    plot_hybrid_html = os.path.join(results_folder, f"{dataset_name}_test_plot_hybrid.html")
    fig.write_html(plot_hybrid_html)
    print(f"  ✓ Gràfica Test HÍBRID guardada a: {plot_hybrid_html}")

    # ───────────────────────────────────────────────────────────
    # 13) PREDICCIÓ AUTOREGRESSIVA FUTURS 10 DIES AMB HÍBRID
    # ───────────────────────────────────────────────────────────
    df_future = df.copy().reset_index(drop=True)
    recompute_indicators(df_future)

    last_sequence = X_test_scaled[-1].copy().reshape(1, N_STEPS, len(FEATURE_COLUMNS))
    future_preds_lstm = []
    future_preds_hybrid = []
    future_dates = pd.bdate_range(start=df_future['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)

    for date in future_dates:
        # 13.1) Pred LSTM -> real
        y_pred_scaled = model.predict(last_sequence, verbose=0)[0][0]
        y_pred_real = scaler_y.inverse_transform([[y_pred_scaled]])[0][0]
        future_preds_lstm.append(y_pred_real)

        # 13.2) Afegeix nova fila per a recompute_indicators
        prev = df_future.iloc[-1]
        new_row = {
            'Date':        date,
            'Open':        prev['Close'],
            'High':        y_pred_real,
            'Low':         y_pred_real,
            'Volume':      prev['Volume'],
            'Close':       y_pred_real,
            'EMA_7':       np.nan,
            'EMA_40':      np.nan,
            'MACD':        np.nan,
            'Signal_Line': np.nan,
            'MACD_Hist':   np.nan,
            'RSI':         np.nan,
            'ATR':         np.nan
        }
        df_future.loc[len(df_future)] = new_row
        recompute_indicators(df_future)

        # 13.3) Construir nova seqüència per LSTM
        recent_features = df_future[FEATURE_COLUMNS].iloc[-N_STEPS:].values
        recent_scaled = scaler_X.transform(recent_features)
        last_sequence = recent_scaled.reshape(1, N_STEPS, len(FEATURE_COLUMNS))

        # 13.4) Predicció HÍBRIDA:
        X_last_future = df_future[FEATURE_COLUMNS].iloc[-1].values.reshape(1, -1)  # shape (1, n_features)
        Xgb_feat = np.concatenate([X_last_future, np.array([[y_pred_real]])], axis=1)
        residual_xgb = xgb_model.predict(Xgb_feat)[0]
        y_pred_hybrid = y_pred_real + residual_xgb
        future_preds_hybrid.append(y_pred_hybrid)

        # 13.5) Actualitzar 'Close' per al següent pas
        df_future.at[df_future.index[-1], 'Close'] = y_pred_hybrid

    # ───────────────────────────────────────────────────────────
    # 14) GUARDAR PREDICCIONS FUTURES HÍBRIDES I LSTM EN CSV
    # ───────────────────────────────────────────────────────────
    df_fut_pred = pd.DataFrame({
        "Date": future_dates,
        "Pred_Close_LSTM": future_preds_lstm,
        "Pred_Close_HYBRID": future_preds_hybrid
    })
    fut_csv = os.path.join(results_folder, f"{dataset_name}_future_10days_hybrid.csv")
    df_fut_pred.to_csv(fut_csv, index=False)
    print(f"  ✓ Prediccions futures híbrides guardades a: {fut_csv}")

    # ───────────────────────────────────────────────────────────
    # 15) GRAFICAR HISTÒRIC + PREDICCIONS FUTURES HÍBRIDES
    # ───────────────────────────────────────────────────────────
    fig_future = go.Figure()
    fig_future.add_trace(go.Scatter(
        x=df['Date'], y=df['Close'],
        mode='lines', name='Històric Close', line=dict(color='lightblue')
    ))
    fig_future.add_trace(go.Scatter(
        x=future_dates, y=np.array(future_preds_hybrid),
        mode='lines+markers', name='Predicció futura HÍBRIDA',
        line=dict(color='orange', dash='dash'),
        marker=dict(size=6)
    ))
    fig_future.update_layout(
        title=f"{dataset_name} – Predicció HÍBRIDA Pròxims 10 Dies",
        xaxis_title='Data',
        yaxis_title='Preu Close (USD)',
        template='plotly_white',
        xaxis_rangeslider_visible=True
    )
    fut_hybrid_html = os.path.join(results_folder, f"{dataset_name}_future_plot_hybrid.html")
    fig_future.write_html(fut_hybrid_html)
    print(f"  ✓ Gràfica futura híbrida guardada a: {fut_hybrid_html}")



===== Procesant: Amazon_Stock_Price_output =====
  ✓ Paràmetres LSTM carregats: layers=1, units=512, dropout=0.1, lr=0.001


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  ✓ Pesos LSTM carregats de: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_best_weights.weights.h5
  → LSTM Test MAE=4.5752, RMSE=5.7316, R²=0.9283
  ✓ Mètriques LSTM guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Amazon_Stock_Price_output\Amazon_Stock_Price_output_metrics_LSTM_test.csv
  ✓ XGBoost entrenat sobre residuals (train+val) per a Amazon_Stock_Price_output
  → HÍBRID Test MAE=4.5288, RMSE=5.7489, R²=0.9279
  ✓ Mètriques híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Amazon_Stock_Price_output\Amazon_Stock_Price_output_metrics_hybrid_test.csv
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Amazon_Stock_Price_output\Amazon_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



  ✓ Pesos LSTM carregats de: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_best_weights.weights.h5
  → LSTM Test MAE=51.0807, RMSE=64.9371, R²=0.7725
  ✓ Mètriques LSTM guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_metrics_LSTM_test.csv
  ✓ XGBoost entrenat sobre residuals (train+val) per a Euro_Stoxx_50_Stock_Price_output
  → HÍBRID Test MAE=45.9459, RMSE=57.8606, R²=0.8194
  ✓ Mètriques híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_metrics_hybrid_test.csv
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbri

...

  ✓ Pesos LSTM carregats de: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Indra_Stock_Price_output\Indra_Stock_Price_output_best_weights.weights.h5
  → LSTM Test MAE=234.5538, RMSE=313.4665, R²=0.7487
  ✓ Mètriques LSTM guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Indra_Stock_Price_output\Indra_Stock_Price_output_metrics_LSTM_test.csv
  ✓ XGBoost entrenat sobre residuals (train+val) per a Indra_Stock_Price_output
  → HÍBRID Test MAE=473.6909, RMSE=539.1290, R²=0.2565
  ✓ Mètriques híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Indra_Stock_Price_output\Indra_Stock_Price_output_metrics_hybrid_test.csv
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\Indra_Stock_Price_output\Indra_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



  ✓ Pesos LSTM carregats de: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_best_weights.weights.h5
  → LSTM Test MAE=2.1416, RMSE=2.6764, R²=0.6456
  ✓ Mètriques LSTM guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\P&G_Stock_Price_output\P&G_Stock_Price_output_metrics_LSTM_test.csv
  ✓ XGBoost entrenat sobre residuals (train+val) per a P&G_Stock_Price_output
  → HÍBRID Test MAE=3.0134, RMSE=3.5953, R²=0.3604
  ✓ Mètriques híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\P&G_Stock_Price_output\P&G_Stock_Price_output_metrics_hybrid_test.csv
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\P&G_Stock_Price_output\P&G_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\P&G_

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 



  ✓ Pesos LSTM carregats de: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_best_weights.weights.h5
  → LSTM Test MAE=71.1080, RMSE=90.2894, R²=0.8303
  ✓ Mètriques LSTM guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\S&P500_Stock_Price_output\S&P500_Stock_Price_output_metrics_LSTM_test.csv
  ✓ XGBoost entrenat sobre residuals (train+val) per a S&P500_Stock_Price_output
  → HÍBRID Test MAE=99.7193, RMSE=116.6878, R²=0.7165
  ✓ Mètriques híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\S&P500_Stock_Price_output\S&P500_Stock_Price_output_metrics_hybrid_test.csv
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRID\S&P500_Stock_Price_output\S&P500_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_Predict

Model híbrid correcte - canvis realitzats en l'estructura del codi

In [2]:
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
import xgboost as xgb

# ───────────────────────────────────────────────────────────────
# CONFIGURATION
# ───────────────────────────────────────────────────────────────
BASE_DIR = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock"
DATA_DIR = os.path.join(BASE_DIR, "Conjunt de dades Preprocessades", "Datasets")
LSTM_WEIGHTS_DIR = os.path.join(BASE_DIR, "LSTM", "RESULTATS_LSTM")
OUTPUT_DIR = os.path.join(BASE_DIR, "LSTM", "RESULTATS_HIBRIDS")
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATASETS = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv",
]

# Hyperparameters
N_STEPS = 30
FEATURE_COLUMNS = [
    "Open", "High", "Low", "Volume",
    "EMA_7", "EMA_40", "MACD", "Signal_Line",
    "MACD_Hist", "RSI", "ATR"
]
TARGET_COLUMN = "Close"
TEST_RATIO = 0.10
VAL_RATIO = 0.10
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO

# ───────────────────────────────────────────────────────────────
# FUNCTIONS
# ───────────────────────────────────────────────────────────────

def load_dataset(path):
    df = pd.read_csv(path, parse_dates=["Date"]).sort_values("Date")
    return df.reset_index(drop=True)


def compute_technical_indicators(df):
    close = df[TARGET_COLUMN]
    df["EMA_7"] = close.ewm(span=7, adjust=False).mean()
    df["EMA_40"] = close.ewm(span=40, adjust=False).mean()
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    df["MACD"] = macd
    df["Signal_Line"] = signal
    df["MACD_Hist"] = macd - signal
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df["RSI"] = 100 - (100 / (1 + rs))
    df["ATR"] = (df["High"] - df["Low"]).rolling(window=14).mean()
    return df


def create_sequences(data, n_steps=N_STEPS):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :-1])
        y.append(data[i, -1])
    return np.array(X), np.array(y)


def build_and_compile_lstm(input_shape, params):
    model = Sequential()
    for i in range(params['n_layers']):
        return_seq = i < params['n_layers'] - 1
        layer_args = dict(units=params['units'], return_sequences=return_seq)
        if i == 0:
            model.add(LSTM(**layer_args, input_shape=input_shape))
        else:
            model.add(LSTM(**layer_args))
        model.add(Dropout(params['dropout']))
    model.add(Dense(1, activation="linear"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(loss="huber", optimizer=optimizer, metrics=["mae"])
    return model


def evaluate_model(model, X, y, scaler):
    y_pred = model.predict(X, verbose=0)
    y_true = scaler.inverse_transform(y.reshape(-1, 1)).flatten()
    y_pred_real = scaler.inverse_transform(y_pred).flatten()
    return {
        'MAE': mean_absolute_error(y_true, y_pred_real),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred_real)),
        'R2': r2_score(y_true, y_pred_real),
        'y_true': y_true,
        'y_pred': y_pred_real
    }


def train_xgb_on_residuals(X_feats, residuals):
    np.random.seed(42)
    idx = np.arange(len(X_feats))
    np.random.shuffle(idx)
    split = int(len(idx) * 0.8)
    train_idx, val_idx = idx[:split], idx[split:]

    X_train_xgb = X_feats[train_idx]
    y_train_xgb = residuals[train_idx]
    X_val_xgb = X_feats[val_idx]
    y_val_xgb = residuals[val_idx]

    xgb_reg = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    xgb_reg.fit(
        X_train_xgb, y_train_xgb,
        eval_set=[(X_val_xgb, y_val_xgb)],
        verbose=False
    )
    return xgb_reg


def plot_results(dates, true_vals, lstm_preds, hybrid_preds, title, output_path):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=true_vals, mode='lines', name='Real'))
    fig.add_trace(go.Scatter(x=dates, y=lstm_preds, mode='lines', name='LSTM Prediction'))
    fig.add_trace(go.Scatter(x=dates, y=hybrid_preds, mode='lines', name='Hybrid Prediction'))
    fig.update_layout(
        title=title,
        xaxis_title='Date', yaxis_title='Close Price',
        template='plotly_dark', xaxis_rangeslider_visible=True
    )
    fig.write_html(output_path)


def main():
    for file in DATASETS:
        dataset_name = os.path.splitext(file)[0]
        print(f"Processing {dataset_name}...")

        # Create a subdirectory for this dataset's outputs
        dataset_output_dir = os.path.join(OUTPUT_DIR, dataset_name)
        os.makedirs(dataset_output_dir, exist_ok=True)

        # Load and preprocess
        df = load_dataset(os.path.join(DATA_DIR, file))
        df = compute_technical_indicators(df).dropna().reset_index(drop=True)

        # Prepare sequences and splits
        arr = df[FEATURE_COLUMNS + [TARGET_COLUMN]].values
        X_all, y_all = create_sequences(arr)
        n_samples = len(X_all)
        train_end = int(n_samples * TRAIN_RATIO)
        val_end = train_end + int(n_samples * VAL_RATIO)

        X_train, y_train = X_all[:train_end], y_all[:train_end]
        X_val, y_val = X_all[train_end:val_end], y_all[train_end:val_end]
        X_test, y_test = X_all[val_end:], y_all[val_end:]

        # Scaling
        scaler_X = MinMaxScaler().fit(X_all.reshape(-1, X_all.shape[-1]))
        scaler_y = MinMaxScaler().fit(y_all.reshape(-1, 1))
        X_train_scaled = scaler_X.transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_val_scaled = scaler_X.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
        X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
        y_train_scaled = scaler_y.transform(y_train.reshape(-1, 1))
        y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1))
        y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

        # Load LSTM
        param_file = os.path.join(LSTM_WEIGHTS_DIR, dataset_name, f"{dataset_name}_best_params.json")
        weights_file = os.path.join(LSTM_WEIGHTS_DIR, dataset_name, f"{dataset_name}_best_weights.weights.h5")
        with open(param_file) as pf:
            params = json.load(pf)
        lstm = build_and_compile_lstm((N_STEPS, len(FEATURE_COLUMNS)), params)
        lstm.load_weights(weights_file)

        # Evaluate LSTM on test
        lstm_metrics = evaluate_model(lstm, X_test_scaled, y_test_scaled, scaler_y)
        print(f"LSTM Test → MAE: {lstm_metrics['MAE']:.4f}, RMSE: {lstm_metrics['RMSE']:.4f}, R2: {lstm_metrics['R2']:.4f}")

        # Hybrid training on residuals
        train_val_scaled = np.concatenate([X_train_scaled, X_val_scaled])
        y_train_val_scaled = np.concatenate([y_train_scaled, y_val_scaled])
        y_pred_train_val = lstm.predict(train_val_scaled)
        true_train_val = scaler_y.inverse_transform(y_train_val_scaled).flatten()
        pred_train_val = scaler_y.inverse_transform(y_pred_train_val).flatten()
        residuals = true_train_val - pred_train_val

        raw_arr = df[FEATURE_COLUMNS].values[N_STEPS:]
        xgb_feats = np.hstack([raw_arr[:len(residuals)], pred_train_val.reshape(-1, 1)])
        xgb_model = train_xgb_on_residuals(xgb_feats, residuals)

        # Test hybrid
        lstm_test_pred = scaler_y.inverse_transform(lstm.predict(X_test_scaled)).flatten()
        raw_test = df[FEATURE_COLUMNS].values[N_STEPS + val_end:]
        xgb_test_feats = np.hstack([raw_test, lstm_test_pred.reshape(-1, 1)])
        hybrid_preds = lstm_test_pred + xgb_model.predict(xgb_test_feats)
        true_test = df[TARGET_COLUMN].values[N_STEPS + val_end:]

        mae_h = mean_absolute_error(true_test, hybrid_preds)
        rmse_h = np.sqrt(mean_squared_error(true_test, hybrid_preds))
        r2_h = r2_score(true_test, hybrid_preds)
        print(f"Hybrid Test → MAE: {mae_h:.4f}, RMSE: {rmse_h:.4f}, R2: {r2_h:.4f}")

        # Save metrics CSV
        metrics_df = pd.DataFrame([{
            **{'Dataset': dataset_name},
            **{f"{k}_LSTM": v for k, v in lstm_metrics.items() if k in ['MAE','RMSE','R2']},
            **{'MAE_HYBRID': mae_h, 'RMSE_HYBRID': rmse_h, 'R2_HYBRID': r2_h}
        }])
        metrics_df.to_csv(os.path.join(dataset_output_dir, f"{dataset_name}_metrics.csv"), index=False)

        # Plot and save hybrid test comparison
        start_test_idx = N_STEPS + val_end
        dates_test = df['Date'].iloc[start_test_idx:start_test_idx + len(true_test)].reset_index(drop=True)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=dates_test, y=true_test, mode='lines', name='Real (Close)'))
        fig.add_trace(go.Scatter(x=dates_test, y=lstm_test_pred, mode='lines', name='Pred LSTM', line=dict(dash='dash')))
        fig.add_trace(go.Scatter(x=dates_test, y=hybrid_preds, mode='lines', name='Pred HÍBRIDA', line=dict(dash='dot')))
        fig.update_layout(
            title=f"{dataset_name} – Real vs Prediccions (Test, LSTM i HÍBRIDA)",
            xaxis_title='Data', yaxis_title='Preu (Close)',
            template='plotly_white', xaxis_rangeslider_visible=True
        )
        hybrid_html = os.path.join(dataset_output_dir, f"{dataset_name}_test_plot_hybrid.html")
        fig.write_html(hybrid_html)
        print(f"  ✓ Gràfica Test HÍBRID guardada a: {hybrid_html}")

        # Future 10-day autoregressive hybrid prediction
        df_future = df.copy().reset_index(drop=True)
        recompute_indicators(df_future)

        last_sequence = X_test_scaled[-1].copy().reshape(1, N_STEPS, len(FEATURE_COLUMNS))
        future_preds_lstm = []
        future_preds_hybrid = []
        future_dates = pd.bdate_range(start=df_future['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)

        for date in future_dates:
            # 13.1) Pred LSTM -> real
            y_pred_scaled = lstm.predict(last_sequence, verbose=0)[0][0]
            y_pred_real = scaler_y.inverse_transform([[y_pred_scaled]])[0][0]
            future_preds_lstm.append(y_pred_real)

            # 13.2) Afegeix nova fila per a recompute_indicators
            prev = df_future.iloc[-1]
            new_row = {
                'Date':        date,
                'Open':        prev['Close'],
                'High':        y_pred_real,
                'Low':         y_pred_real,
                'Volume':      prev['Volume'],
                'Close':       y_pred_real,
                'EMA_7':       np.nan,
                'EMA_40':      np.nan,
                'MACD':        np.nan,
                'Signal_Line': np.nan,
                'MACD_Hist':   np.nan,
                'RSI':         np.nan,
                'ATR':         np.nan
            }
            df_future.loc[len(df_future)] = new_row
            recompute_indicators(df_future)

            # 13.3) Construir nova seqüència per LSTM
            recent_features = df_future[FEATURE_COLUMNS].iloc[-N_STEPS:].values
            recent_scaled = scaler_X.transform(recent_features)
            last_sequence = recent_scaled.reshape(1, N_STEPS, len(FEATURE_COLUMNS))

            # 13.4) Predicció HÍBRIDA:
            X_last_future = df_future[FEATURE_COLUMNS].iloc[-1].values.reshape(1, -1)  # shape (1, n_features)
            Xgb_feat = np.concatenate([X_last_future, np.array([[y_pred_real]])], axis=1)
            residual_xgb = xgb_model.predict(Xgb_feat)[0]
            y_pred_hybrid = y_pred_real + residual_xgb
            future_preds_hybrid.append(y_pred_hybrid)

            # 13.5) Actualitzar 'Close' per al següent pas
            df_future.at[df_future.index[-1], 'Close'] = y_pred_hybrid

        # ───────────────────────────────────────────────────────────
        # 14) GUARDAR PREDICCIONS FUTURES HÍBRIDES I LSTM EN CSV
        # ───────────────────────────────────────────────────────────
        df_fut_pred = pd.DataFrame({
            "Date": future_dates,
            "Pred_Close_LSTM": future_preds_lstm,
            "Pred_Close_HYBRID": future_preds_hybrid
        })
        fut_csv = os.path.join(dataset_output_dir, f"{dataset_name}_future_10days_hybrid.csv")
        df_fut_pred.to_csv(fut_csv, index=False)
        print(f"  ✓ Prediccions futures híbrides guardades a: {fut_csv}")

        # Plot and save historical + future hybrid
        fig_fut = go.Figure()
        fig_fut.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Històric Close', line=dict(color='lightblue')))
        fig_fut.add_trace(go.Scatter(x=future_dates, y=future_preds_hybrid, mode='lines+markers', name='Pred futura HÍBRIDA', line=dict(color='orange', dash='dash'), marker=dict(size=6)))
        fig_fut.update_layout(
            title=f"{dataset_name} – Predicció Pròxims 10 Dies (HÍBRID: LSTM + XGBOOST)",
            xaxis_title='Data', yaxis_title='Preu (Close)', template='plotly_white', xaxis_rangeslider_visible=True
        )
        fut_html = os.path.join(dataset_output_dir, f"{dataset_name}_future_plot_hybrid.html")
        fig_fut.write_html(fut_html)
        print(f"  ✓ Gràfica futura híbrida guardada a: {fut_html}")

if __name__ == '__main__':
    main()


Processing Amazon_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



LSTM Test → MAE: 3.6188, RMSE: 4.6093, R2: 0.9534
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Hybrid Test → MAE: 3.2784, RMSE: 4.2841, R2: 0.9597
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Amazon_Stock_Price_output\Amazon_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_plot_hybrid.html
Processing Euro_Stoxx_50_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



LSTM Test → MAE: 68.0060, RMSE: 85.1416, R2: 0.5995
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Hybrid Test → MAE: 25.9500, RMSE: 32.4365, R2: 0.9419
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_plot_hybrid.html
Processing Google_Stock_Price_output...


...

LSTM Test → MAE: 233.7392, RMSE: 295.7193, R2: 0.7778
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Hybrid Test → MAE: 171.6424, RMSE: 226.0883, R2: 0.8701
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Indra_Stock_Price_output\Indra_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Indra_Stock_Price_output\Indra_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Indra_Stock_Price_output\Indra_Stock_Price_output_future_plot_hybrid.html
Processing P&G_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



LSTM Test → MAE: 1.8727, RMSE: 2.3827, R2: 0.7218
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Hybrid Test → MAE: 1.1756, RMSE: 1.5143, R2: 0.8876
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\P&G_Stock_Price_output\P&G_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\P&G_Stock_Price_output\P&G_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\P&G_Stock_Price_output\P&G_Stock_Price_output_future_plot_hybrid.html
Processing S&P500_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 



LSTM Test → MAE: 81.7329, RMSE: 101.3684, R2: 0.7772
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Hybrid Test → MAE: 73.3193, RMSE: 86.9410, R2: 0.8361
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\S&P500_Stock_Price_output\S&P500_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_plot_hybrid.html
